#### Número de grupo: 
  - 5

#### Nombre de los integrantes del grupo:
  - Gema Blanco Núñez
  
  - Diego Alejandro Rodríguez Pereira

# Práctica 1

> __Fecha de entrega: 11 de abril de 2021__


## Parte 2: similitud semántica

Una de las grandes ventajas de las representaciones estructuradas es que podemos aprovechar su estructura para calcular similitudes semánticas entre las entidades. En esta ocasión vamos a cacular la similitud entre dos conceptos como:

$$Sim(A, B) = \frac{\delta(root, C)}{\delta(root, C) + \delta(C, A) + \delta(C, B)}$$

siendo:

- $\delta(X, Y)$ el __mínimo__ número de aristas que conecta A y B, siendo A más general que B.
- $C = LCS(A, B)$ el concepto más específico de la jerarquía que es más general que A y B (_least common subsummer_).

La idea tras esta similitud queda reflejada en la siguiente imagen:

<img src="sim.png" alt="Similitud" style="width: 300px;"/>

En la práctica pueden existir distintos conceptos C que cumplen la definición de _least common subsummer_ de A y B por lo que es necesario definir cuál de ellos vamos a utilizar. En nuestro caso seleccionaremos __uno de los que maximiza el valor de similitud__. 

### 1) Obtener la taxonomía con la que vamos a trabajar

Utiliza el [punto el acceso](https://query.wikidata.org/) SPARQL de Wikidata para ejecutar una consulta que devuelva todos los pares de entidades $(x, y)$ tal que $x$ es subconcepto directo de $y$ y ambos son un tipos de [instrumentos musicales (Q34379)](https://www.wikidata.org/wiki/Q34379). Debes recuperar tantos las URIs de la entidades como sus etiquetas.

Escribe en la siguiente celda la consulta que has utilizado comentada adecuadamente.

#### Fecha consulta: 01/04/2021

#### Hemos Obtenido:


<html><head><meta charset="utf-8"></head><body><table><thead><tr><th>count</th></tr></thead><tbody><tr><td>4543</td></tr></tbody></table></body></html>

A continuación descarga todas las respuestas en formato _Archivo JSON_ y guardalo en el mismo directorio de la práctica.

_Nota: en el momento de realizar esta práctica obtuve 4727 resultados pero el número puede variar al ser Wikidata una base de conocimiento dinámica._

### 2) Cargar la taxonomía en memoria

Vamos a cargar la taxonomía de clases en memoria para poder operar con ella. Representaremos la jerarquía de lcases mediantes las siguientes estructuras:

- Un diccionario que asocia a cada identificador su etiqueta (por ejemplo 'Q34379' -> 'musical instrument')
- Un diccionario que asocia cada clase con sus subclases directas (por ejemplo 'Q695269' -> {'Q25630013', 'Q3388256', 'Q524526', 'Q846109', 'Q960389'} )
- Un diccionario que asocia cada clase con sus superclases directas (por ejemplo 'Q34379' -> {'Q1879241', 'Q54820129'} )

Tienes libertad para elegir cómo quieres representar la taxonomía en Python:

- Puedes usar una clase. En ese caso tendrás que ir añadiendo métodos a la clase para completar cada uno de los apartados de la práctica. Escribe el código de la clase en una única celda y utiliza los métodos que necesites en cada uno de los apartados.
- Puedes usar 3 variables globales para representar la taxonomía. En ese caso deberás escribir las operaciones como funciones en cada uno de los apartados de la práctica.

En cualquier caso recuerda documentar adecuadamente el código y trata de que sea sencillo de entender.

Crea una operación _load_ que reciba el nombre del fichero json y cargue el grafo en memoria usando las estructuras anteriores.

```python
import json

with open(filename) as f:
    data = json.load(f)
```

In [2]:
import json

class Taxonomy():
    
    def __init__(self):
        #Creamos 3 diccionarios:
        #Diccionario identificadores (clave:identificador, valor:etiqueta)
        self.idDict = dict()
        #Diccionario de clases de diccionarios de subclases clave:identificador valor:Diccionario(clave:identificador, valor:etiqueta)
        #Contiene un diccionario con las clases, y dentro tiene otro diccionario con las clases que son sublclases 
        #de la clase principal
        self.subClassDict = dict(set())
        #Diccionario de clases de diccionarios de superclases clave:identificador valor:Diccionario(clave:identificador, valor:etiqueta)
        #contiene un diccionario con las clases, y dentro tiene otro diccionario con las clases que son superlclases 
        #e la clase principal
        self.superClassDict = dict(set())
    
    #########################
    #Apartado 2
    #########################
    
    def load(self, filename):
        """ Esta función carga el fichero json para clasificar los instrumentos
        en cada uno de los 3 diccionarios"""
        
        #Abrimos y cargamos el fichero Json
        with open(filename, encoding='utf-8') as f:
            data = json.load(f)
        
        #Por cada valor que este en el fichero
        #obtenemos la clase Y junto con su etiqueta
        # y obtenemos la clase X junto con su etiqueta
        for instrument in data:
            
            y_aux = instrument['y']
            y = y_aux.replace('http://www.wikidata.org/entity/', '')
            x_aux = instrument['x']
            x = x_aux.replace('http://www.wikidata.org/entity/', '')
            yLabel = instrument['yLabel']
            xLabel = instrument['xLabel']
            
            #Añadimos los intrumentos a los 3 diccionarios
            self.addID(y, yLabel, x, xLabel)
            self.addSubClass(y, yLabel, x, xLabel)
            self.addSuperClass(y, yLabel, x, xLabel)
    
    
    def addID(self, y, yLabel, x, xLabel):
        """Añade las etiquetas al diccionario de identificadores"""
        
        self.idDict[y] = yLabel
        self.idDict[x] = xLabel
    
    
    def addSubClass(self, y, yLabel, x, xLabel):
        """Añade la clase x como subclase de y"""
    
        #Si ya existe la clase Y en el diccionario, simplemente añadimos a la clase X
        #al conjuto (set) de subclases de Y
        #Si estamos introduciento la clase Y por primera vez,
        #la añadimos al diccionario junto con la creacción del conjunto (set) de elementos
        #que serán subclase de Y
        if (y in self.subClassDict):
            self.subClassDict[y].add(x)
        else:
            self.subClassDict[y] = {x}
            
        if (x not in self.subClassDict):
            self.subClassDict[x] = set()
    
    def addSuperClass(self, y, yLabel, x, xLabel):
        """Añade la clase y como superclase de x"""
        
        #Si ya existe la clase X en el diccionario, simplemente añadimos a la clase Y
        #al conjuto (set) de superclases de X
        #Si estamos introduciento la clase X por primera vez,
        #la añadimos al diccionario junto con la creacción del conjunto (set) de elementos
        #que serán subclase de Y
        if (x in self.superClassDict):
            self.superClassDict[x].add(y)
        else:
            self.superClassDict[x] = {y}

    
    #########################
    #Apartado 3
    #########################
    
    def _print_tree_(self, clase, i, numNiveles):
        """Función que imprime el subárbol de la taxonomía
        Parámetros
        clase: es la clase/identidificador a partir del cúal queremos imprimir
        i: el nivel por el que va el subárbol, se inicializa a 0
        numNiveles: el númerod de niveles que queremos obtener en el subárbol, si este
        nivel sobrepasa el de la taxonomía, imprime hasta donde alcance la taxonomía"""
    
        #Primero imprimimos la clase inicial
        #Luego obtenemos las subclases directas de la clase inicial
        #Y llamamos recursivamente a la función para imprimir los siguientes niveles
        s = ""
        for j in range(i):
            s += "  "
        print(s, i," ", self.idDict[clase], ' (', clase, ')', sep='')
        
        if (i < numNiveles - 1):
            for instrument in self.subClassDict[clase]:
                self._print_tree_(instrument, i + 1, numNiveles)
    
    
    #########################
    #Apartado 4
    #########################
                
    def getAllSuperClasses(self, clase, listSuperClasses):
        """Función que obtiene todas las superclases de una clase"""
        
        #Primero verificamos que la clase tenga superclases (que serán todas menos 'musical instruments' Q34379)
        #Luego añadimos a cada instrumento a la lista
        #Y llamamos recursivamente por si el instrumento tiene a su vez otras superclases
        if (clase in self.superClassDict):
            for instrument in self.superClassDict[clase]:
                listSuperClasses.append(instrument)
                self.getAllSuperClasses(instrument, listSuperClasses)
    
    
    
    def getAllSubClasses(self, clase, listSubClasses):
        """Función que obtiene todas las subclases de una clase"""
        
        #Primero verificamos que la clase tenga subclase(s)
        #Luego añadimos a cada instrumento a la lista
        #Y llamamos recursivamente por si el instrumento tiene a su vez otras subclase(s)
        if (clase in self.subClassDict):
            for instrument in self.subClassDict[clase]:
                listSubClasses.append(instrument)
                self.getAllSubClasses(instrument, listSubClasses)
                
                
    def LCS(self, classA, classB):
        """Least Common Subsummer"""
        
        #Primero obtenemos dos listas:
        #para las superclases de A y para las superclases de B
        listSuperClassesA = list() 
        listSuperClassesA.append(classA)
        self.getAllSuperClasses(classA, listSuperClassesA)
        listSuperClassesB = list()
        listSuperClassesB.append(classB)
        self.getAllSuperClasses(classB, listSuperClassesB)
        
        #Luego creamos una lista con la solución
        #Por cada instrumento de A verificamos si este aparece en B.
        #Si aparecen quiere decir que comparten ese mismo instrumento/clase/nodo
        #Por tanto sólo nos queda verificar que esa clase común no sea más general que
        #algunas de las clases ya puestas en como solución. (Ya que queremos la clase/concepto más específico)
        #Nota: cabe destacar que las listas están ordenadas por especificidad
        sol = list()
        for instrument in listSuperClassesA:
            if (instrument in listSuperClassesB):
                if (self.verify(instrument, sol)):
                    sol.append(instrument)
        
        return (sol)
    
    
    def verify(self, instrument, solution):
        """Función que verifica que una clase no sea más
        específica que otra"""
        
        listSubClassesA = list()
        listSubClassesA.append(instrument)
        self.getAllSubClasses(instrument, listSubClassesA)
        
        #Si algunas de las clases de la solución aparece como subclase
        #del intrumento que queremos añadir, quiere decir que este instrumento
        #es más general y por tanto no podemos añadirlo.
        encontrado = False
        for sol in solution:
            if sol in listSubClassesA:
                encontrado = True
        
        return (not encontrado)
    
    
    #########################
    #Apartado 5
    #########################
    
    """Función que encuentra el camino mínimo entre A y B"""
    def path(self, A, B):
        sol = list()
        self.minDistance = 0
        self.minPath = list()
        self.minPath.append(A)
        
        if(A != B): #Encontrar camino hasta B desde A
            path = self.minPath.copy()
            distance = 1
            self.find_path(A, B, distance, path) 
            
        for i in self.minPath:
            sol.append(self.idDict[i]+' ('+i+')')
       
        return sol
                

    """Función recursiva auxiliar para encontrar el camino mínimo"""            
    def find_path(self, A, B, distance, path):
        
        if A in self.subClassDict:
            for i in self.subClassDict[A]: #Recorrido en anchura
                path.append(i) 
                
                #Si encontramos a B devolvemos el camino con la distancia
                #Si todavia no encontramos a B seguimos buscando en profundidad
                if i == B: 
                    if self.minDistance == 0 or distance < self.minDistance:
                        self.minPath = path.copy()
                        self.minDistance = distance

                else:  #Recorrido en profundidad
                    self.find_path(i, B, distance+1, path)
                    
                path.remove(i)
    
    
    #########################
    #Apartado 6
    #########################
    
    #Comentarios abajo del def
    def similarity(self, A, B):
        """Función que calcula la similitud entre dos conceptos A y B.
        Dicha similitud se calcula con la fórmula al inicio de este documento."""   
        C = self.LCS(A,B)
        root = 'Q34379'
        for i in C:
            xPath = self.path(root,i) #Camino minimo entre la raiz y C
            x = self.minDistance
            yPath = self.path(i,A)    #Camino minimo entre C y A
            y = self.minDistance
            zPath = self.path(i,B)    #Camino minimo entre C y B
            z = self.minDistance

        
        #Aplicar formula para obtener la similitud entre A y B
        sim = x / (x + y + z)
        
        #Descomentar para mostrar los caminos minimos
        print(xPath,yPath,zPath)
        
        return sim
    
    

#### Crear la taxonomia

In [ ]:
t = Taxonomy()

#### Cargar datos de la consulta 1

In [ ]:
t.load('query.json')

### 3) Imprimir un subárbol de la taxonomía

Crea una operación _print_tree_ que imprimir la jerarquía de clases a partir de un concepto y hasta un nivel de profundidad determinado.

Por ejemplo, a continuación podemos ver el principio de la jerarquía de [voces](https://www.wikidata.org/wiki/Q17172850) con 3 niveles de profundidad:

```
0 voz (Q17172850)
  1 operatic vocal (Q101436564)
  1 alto (Q6983813)
   2 mezzosoprano ligera (Q6012300)
   2 boy alto (Q53395277)
   2 alto castrato (Q53395016)
   2 contralto (Q37137)
  1 contralto (Q37137)
   2 contralto cómica (Q5785182)
   2 lyric contralto (Q54635214)
   2 Tenorino (Q6141663)
   2 contralto de coloratura (Q54635184)
   2 deep contralto (Q54635335)
   2 contralto dramática (Q5785183)
  1 bajo (Q27911)
   2 heavy acting bass (Q54636271)
   2 bajo profundo (Q2532487)
   2 bajo buffo (Q1002146)
   ...
```

Como ocurre en todas las grandes bases de conocimiento, dentro de Wikidata hay información que no ha sido bien introducida o está mal clasificada. ¿Puedes encontrar algún ejemplo concreto dentro de la jerarquía de instrumentos?

#### Algunas de los conceptos que hemos encontrado

Este es el caso en el que se añadió una clase al lenguaje en español, ya existiendo otra versión en otros lenguajes. Y como estos dos identificadores están clasificados de forma distinta, el 'Q54634726' aparece como subclase de mezzo-soprano (Q186506) y 'Q6012300' aparece como subclase de alto (Q6983813) entonces para Wikidata son distintos

mezzosoprano ligera (Q54634726)

mezzosoprano ligera (Q6012300)

#### Jerarquía Voces con 3 niveles de profundidad

In [ ]:
t._print_tree_('Q17172850', 0, 3)

#### Jerarquía Viola Eléctrica

In [ ]:
t._print_tree_('Q15336282', 0, 100)

#### Jerarquía Instrumento Musical

In [ ]:
t._print_tree_('Q34379', 0, 100)

### 4) Obtener los LCS

Crea una operación _lcs_ que devuelva todos los LCS de dos conceptos determinados. Recuerda que un concepto C es LCS(A, B) si es más general que ambos y no se puede especializar más sin dejar de serlo.

Para implementarlo seguramente te resulte útil tener otro método que devuelva todos los conceptos más generales que uno dado. _Pista: es fácil de implementar usando operaciones entre conjuntos_. 

Ejemplos:

```
mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
LCS('Q6012297', 'Q54634726') = {'Q186506'}

grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
LCS('Q5885030', 'Q6012300') = {'Q17172850'}

tenor (Q27914)
LCS('Q27914', 'Q27914') = {'Q27914'}

viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603}
```

#### LCS(mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726)) = mezzosoprano (Q186506)

In [ ]:
t.LCS('Q6012297','Q54634726')

#### LCS(grave (Q5885030), mezzosoprano ligera (Q6012300)) = voz (Q17172850)

In [ ]:
t.LCS('Q5885030','Q6012300')

#### LCS(tenor (Q27914), tenor (Q27914))

In [ ]:
t.LCS('Q27914','Q27914')

#### LCS(viola eléctrica (Q15336282), bajo eléctrico (Q64166304)) = {instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)}

In [ ]:
t.LCS('Q15336282','Q64166304')

### 5) Obtener caminos mínimos

Crea una operación _path_ que calcule el camino mínimo entre dos conceptos A y B siendo A más o igual de general que B. Como la taxonomía no tiene ciclos puedes implementarlo como una búsqueda en profunidad. Ten en cuenta que los caminos sólo pueden contener conceptos más específicos o iguales a A y más generales o iguales a B.

Ejemplos:

```
path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

path('Q27914', 'Q27914') = [tenor (Q27914)]

path('Q34379', 'Q55724840') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
 ```

#### path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

In [ ]:
t.path('Q186506', 'Q54634726')

#### path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

In [ ]:
t.path('Q17172850', 'Q6012300')

#### path('Q27914', 'Q27914') = [tenor (Q27914)]

In [ ]:
t.path('Q27914', 'Q27914')

#### path('Q34379', 'Q105738') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]

In [ ]:
t.path('Q34379', 'Q55724840')

### 6) Calcular la similitud

Implementa una operación _similarity_ que calcule la similtud entre dos conceptos. Debe devolver tanto el valor númerico de similitud como los caminos desde la raiz al LCS y desde el LCS a cada uno de los dos conceptos.

Ten en cuenta que debes usar un LCS que maximice el valor de similitud. Si la información de Wikidata no ha cambiado, los valores de similitud deberían coincidir con los que aparecen en los ejemplos pero los caminos no tienen por qué. Y recuerda que no es lo mismo el números de aristas de un camino que el número de nodos del camino.

Ejemplos:

```
similarity('Q6012297', 'Q54634726')
0.5
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q186506', 'Q54634726')
0.6666666666666666
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q27914', 'Q27914')
1.0
[instrumento musical (Q34379), voz (Q17172850), high voice (Q98116969), tenor (Q27914)]
[tenor (Q27914)]
[tenor (Q27914)]

similarity('Q76239', 'Q78987')
0.42857142857142855
[instrumento musical (Q34379), cordófono (Q1051772), instrumento de cuerda (Q1798603), instrumento de cuerda pulsada (Q230262)]
[instrumento de cuerda pulsada (Q230262), cítara (Q76239)]
[instrumento de cuerda pulsada (Q230262), plucked necked box lutes (Q57306162), guitarra (Q6607), guitarra eléctrica (Q78987)]
```

#### similarity('Q6012297', 'Q54634726')

In [ ]:
t.similarity('Q6012297', 'Q54634726')

#### similarity('Q186506', 'Q54634726')

In [ ]:
t.similarity('Q186506', 'Q54634726')

#### similarity('Q27914', 'Q27914')

In [ ]:
t.similarity('Q27914', 'Q27914')

#### similarity('Q76239', 'Q78987')

In [ ]:
t.similarity('Q76239', 'Q78987')

### 7) Análisis de las similitudes

Calcula la similitud 2 a 2 de los siguientes instrumentos y explica razonadamente si los valores obtenidos tienen sentido de acuerdo a tu intuición sobre si se parecen o no.

```
piano (Q5994), guitarra (Q6607), guitarra eléctrica (Q78987), flauta (Q11405), trompeta (Q8338)
```

### Piano

#### Piano - Piano (PP)

In [ ]:
t.similarity('Q5994', 'Q5994')

#### Piano - Guitarra (PG)

In [ ]:
t.similarity('Q5994', 'Q6607')

#### Piano - Guitarra Eléctrica (PGE)

In [ ]:
t.similarity('Q5994', 'Q78987')

#### Piano - Flauta (PF)

In [ ]:
t.similarity('Q5994', 'Q11405')

#### Piano - Trompeta (PT)

In [ ]:
t.similarity('Q5994', 'Q8338')

### Guitarra

#### Guitarra - Guitarra (GGE)

In [ ]:
t.similarity('Q6607', 'Q6607')

#### Guitarra - Guitarra Eléctrica (GGE)

In [ ]:
t.similarity('Q78987', 'Q6607')

#### Guitarra - Flauta (GF)

In [ ]:
t.similarity('Q6607', 'Q11405')

#### Guitarra - Trompeta (GT)

In [ ]:
t.similarity('Q6607', 'Q8338')

### Guitarra Eléctrica

#### Guitarra Eléctrica - Guitarra Eléctrica (GEGE)

In [ ]:
t.similarity('Q78987', 'Q78987')

#### Guitarra Eléctrica - Flauta (GEF)

In [ ]:
t.similarity('Q78987', 'Q11405')

#### Guitarra Eléctrica - Trompeta (GET)

In [ ]:
t.similarity('Q78987', 'Q8338')

### Flauta

#### Flauta - Flauta (FF)

In [ ]:
t.similarity('Q11405', 'Q11405')

#### Flauta - Trompeta (FT)

In [ ]:
t.similarity('Q11405', 'Q8338')

### Trompeta

#### Trompeta - Trompeta (TT)

In [ ]:
t.similarity('Q8338', 'Q8338')

#### Resultados

<table>
  <thead>
    <tr>
      <th>PP</th>
      <th>PG</th>
      <th>PGE</th>
      <th>PF</th>
      <th>PT</th>
      <th>GG</th>
      <th>GGE</th>
      <th>GF</th>
      <th>GT</th>
      <th>GEGE</th>
      <th>GEF</th>
      <th>GET</th>
      <th>FF</th>
      <th>FT</th>
      <th>TT</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1.0</td>
      <td>0.2727272727272727</td>
      <td>0.25</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.8333333333333334</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.4</td>
      <td>1.0</td>
    </tr>
  </tbody>
</table>

## Conclusiones

Como hemos visto en teoría, el cálculo de similitud es simétrico. Por tanto aplicar similitud(Piano, Guitarra) es lo mismo que similitud(Guitarra, Piano)

Para obtener un grado de similitud máximo, se obtiene únicamente haciendo el cálculo consigo mismo, por tanto similitud(Piano, Piano) nos devuelve similitud 1, confirmando de esta manera que son la misma clase.

- Piano:
    Consideramos que las similitudes de Piano con el resto son justas, a pesar de que consideramos que su similitud con Trompeta y Flauta son muy bajas.
    Pensamos así, ya que en el caso con la Guitarra, ambos instrumentos son de cuerda. Mientras que los otros dos son de viento. Con el caso de Guitarra Eléctrica, tiene sentido que tenga menos similitud, ya que se asemeja más a un piano eléctrico que al piano clásico.
    
    
- Guitarra y Guitarra Eléctrica:
    Concordarmos en que su similitud sea bastante alta, ya que Guitarra Eléctrica es subclase directa de Guitarra. Y cuando comparamos estos dos fuera de la taxonomía es lógico que tengan similitud.
    Su similitud con Trompeta y Flauta es 0, ya que sucede como en piano, Guitarra y Guitarra Eléctrica son instrumentos de cuerda, mientras que los otros dos son de viento.
    
    
- Flauta y Trompeta:
    Entre ellos dos guardan una mayor similitud, ya que ambos tienen sus parecidos, como lo es ser intrumento de viento.
    